In [1]:
"""
Urban Heat Risk Analysis - IRIS Level Aggregation
Cell 1: Setup and Imports
"""

# Import necessary libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
import io
from urllib.request import urlopen

# Set up plotting style
plt.style.use('default')
%matplotlib inline

# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [2]:
"""
Cell 2: Load Paris LCZ Data and Add Heat Scores
"""

# Import config (adjust path as needed for your setup)
import sys
sys.path.append('..')
try:
    from config import LCZ_DIR, LCZ_HEAT_MAPPING
except ImportError:
    # Fallback if config not available
    LCZ_HEAT_MAPPING = {
        '1': 9.0, '2': 8.5, '3': 8.5, '4': 8.0, '5': 7.0,
        '6': 6.0, '7': 6.5, '8': 5.0, '9': 4.0, '10': 3.0,
        '11': 2.5, '12': 2.0, '13': 3.0, '14': 2.0, '15': 1.0,
        '16': 1.0, '17': 3.0
    }
    LCZ_DIR = Path('/Users/antoineverhulst/Documents/Project/claude/heat_risk_france/data/raw/lcz')

# Load Paris LCZ data
paris_dir = LCZ_DIR / 'Paris'
shapefile_path = list(paris_dir.glob('LCZ_SPOT_2022_Paris.shp'))[0]

print('Loading Paris LCZ data...')
paris_lcz = gpd.read_file(shapefile_path)
print(f'✅ Loaded {len(paris_lcz):,} LCZ zones')

# Add heat score based on LCZ type
paris_lcz['heat_score'] = paris_lcz['lcz'].map(LCZ_HEAT_MAPPING)

print(f'\n📊 Heat Score Summary:')
print(f'   - Min: {paris_lcz["heat_score"].min():.1f}')
print(f'   - Max: {paris_lcz["heat_score"].max():.1f}')
print(f'   - Mean: {paris_lcz["heat_score"].mean():.2f}')
print(f'   - Median: {paris_lcz["heat_score"].median():.2f}')


Loading Paris LCZ data...
✅ Loaded 156,314 LCZ zones

📊 Heat Score Summary:
   - Min: 0.0
   - Max: 10.0
   - Mean: 3.25
   - Median: 3.00


In [3]:
"""
Cell 3: Load IRIS Administrative Boundaries from data.gouv.fr
"""

import zipfile
import io
from urllib.request import urlopen

# Data.gouv.fr direct download URL for IRIS boundaries
iris_download_url = "https://www.data.gouv.fr/api/1/datasets/r/52c88b33-5328-4e6f-ab8e-fc625794b442"

print("📥 Downloading IRIS boundaries from data.gouv.fr...")
try:
    # Download the file
    response = urlopen(iris_download_url)
    file_bytes = io.BytesIO(response.read())
    
    # Check if it's a zip file
    if response.headers.get('content-type', '').find('zip') != -1 or iris_download_url.endswith('.zip'):
        print("📦 Extracting ZIP archive...")
        with zipfile.ZipFile(file_bytes, 'r') as zip_ref:
            # List files in the zip
            file_list = zip_ref.namelist()
            print(f"✅ Downloaded! Files in archive:")
            for f in file_list[:10]:  # Show first 10 files
                print(f"   - {f}")
            
            # Find shapefile
            shp_files = [f for f in file_list if f.endswith('.shp')]
            if shp_files:
                # Extract all files to temp directory
                temp_dir = Path("./temp_iris")
                temp_dir.mkdir(exist_ok=True)
                zip_ref.extractall(temp_dir)
                
                # Load shapefile
                shp_file = temp_dir / shp_files[0]
                print(f"\n📂 Loading shapefile: {shp_files[0]}")
                iris = gpd.read_file(shp_file)
            else:
                print("❌ No shapefile (.shp) found in archive")
                iris = None
    else:
        # Try loading directly as GeoJSON or other format
        print("💾 File appears to be a single file (not ZIP). Attempting to load directly...")
        iris = gpd.read_file(file_bytes)
    
    if iris is not None:
        print(f"\n✅ Loaded IRIS data successfully!")
        print(f"   - Total IRIS zones: {len(iris):,}")
        print(f"   - Columns: {iris.columns.tolist()}")
        print(f"   - CRS: {iris.crs}")
        
        # Filter for Paris (department code 75)
        # Try different column names for filtering
        paris_filter = None
        
        if 'dep' in iris.columns:
            iris_paris = iris[iris['dep'] == '75'].copy()
            paris_filter = 'dep'
        elif 'insee_com' in iris.columns:
            iris_paris = iris[iris['insee_com'].astype(str).str.startswith('75')].copy()
            paris_filter = 'insee_com'
        else:
            # If no department column found, show available columns
            print("\n⚠️  Could not auto-detect department column. Available columns:")
            for col in iris.columns:
                print(f"   - {col}")
            iris_paris = iris.copy()
            print("\nUsing all data (may not be Paris only)")
        
        print(f"\n✅ Filtered for Paris: {len(iris_paris):,} IRIS zones")
        if paris_filter:
            print(f"   - Filter used: {paris_filter}")
        
except Exception as e:
    print(f"❌ Error downloading IRIS data: {e}")
    print(f"\n⚠️  Alternative: Download manually from:")
    print(f"   {iris_download_url}")
    print("   Then update the iris_data_dir path to your local file")
    iris_paris = None

📥 Downloading IRIS boundaries from data.gouv.fr...
📦 Extracting ZIP archive...
✅ Downloaded! Files in archive:
   - iris.shp
   - iris.shx
   - iris.dbf
   - iris.prj

📂 Loading shapefile: iris.shp

✅ Loaded IRIS data successfully!
   - Total IRIS zones: 5,264
   - Columns: ['dep', 'insee_com', 'nom_com', 'iris', 'code_iris', 'nom_iris', 'typ_iris', 'id', 'geometry']
   - CRS: EPSG:4326

✅ Filtered for Paris: 992 IRIS zones
   - Filter used: dep


In [14]:
"""
Cell 4: Download and Load INSEE Couples/Families/Households Data
"""

# URL for INSEE demographic data
insee_url = 'https://www.insee.fr/fr/statistiques/fichier/8647008/base-ic-couples-familles-menages-2022_csv.zip'

print('📥 Downloading INSEE demographic data...')
try:
    response = urlopen(insee_url)
    zip_buffer = io.BytesIO(response.read())
    
    # Extract the ZIP file
    with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
        # List files in the zip
        file_list = zip_ref.namelist()
        print(f'✅ Downloaded! Files in archive:')
        for f in file_list:
            print(f'   - {f}')
        
        # Find CSV file
        csv_files = [f for f in file_list if f.endswith('.CSV')]
        if csv_files:
            csv_file = csv_files[0]
            print(f'\n📂 Loading CSV: {csv_file}')
            
            # Read CSV with proper encoding
            insee_data = pd.read_csv(
                io.StringIO(zip_ref.read(csv_file).decode('utf-8')),
                dtype={"IRIS": "string", "COM": "string","LAB_IRIS": "string"},
                sep=';'
            )
            print(f'✅ Loaded {len(insee_data):,} records')
            print(f'   - Columns: {insee_data.columns.tolist()}')
            print(f'\n📊 First few rows:')
            print(insee_data.head())
        else:
            print('❌ No CSV file found in archive')
except Exception as e:
    print(f'❌ Error downloading INSEE data: {e}')
    print('\n⚠️  Please ensure you have internet connection or download manually:')
    print(f'   {insee_url}')
    insee_data = None

📥 Downloading INSEE demographic data...
✅ Downloaded! Files in archive:
   - base-ic-couples-familles-menages-2022.CSV
   - meta_base-ic-couples-familles-menages-2022.CSV

📂 Loading CSV: base-ic-couples-familles-menages-2022.CSV
✅ Loaded 49,276 records
   - Columns: ['IRIS', 'COM', 'TYP_IRIS', 'LAB_IRIS', 'C22_MEN', 'C22_MENPSEUL', 'C22_MENHSEUL', 'C22_MENFSEUL', 'C22_MENSFAM', 'C22_MENFAM', 'C22_MENCOUPSENF', 'C22_MENCOUPAENF', 'C22_MENFAMMONO', 'C22_PMEN', 'C22_PMEN_MENPSEUL', 'C22_PMEN_MENHSEUL', 'C22_PMEN_MENFSEUL', 'C22_PMEN_MENSFAM', 'C22_PMEN_MENFAM', 'C22_PMEN_MENCOUPSENF', 'C22_PMEN_MENCOUPAENF', 'C22_PMEN_MENFAMMONO', 'P22_POP15P', 'P22_POP1524', 'P22_POP2554', 'P22_POP5579', 'P22_POP80P', 'P22_POPMEN15P', 'P22_POPMEN1524', 'P22_POPMEN2554', 'P22_POPMEN5579', 'P22_POPMEN80P', 'P22_POP15P_PSEUL', 'P22_POP1524_PSEUL', 'P22_POP2554_PSEUL', 'P22_POP5579_PSEUL', 'P22_POP80P_PSEUL', 'P22_POP15P_MARIEE', 'P22_POP15P_PACSEE', 'P22_POP15P_CONCUB_UNION_LIBRE', 'P22_POP15P_VEUFS', 'P22_

In [23]:
"""
Cell 6: Compute Average Heat Score per IRIS
"""

if iris_paris is not None and paris_lcz is not None:
    print('='*70)
    print('COMPUTING AVERAGE HEAT SCORE PER IRIS')
    print('='*70)
    
    # Reproject LCZ to match IRIS CRS if needed
    if paris_lcz.crs != iris_paris.crs:
        print(f'\n🔄 Reprojecting LCZ data from {paris_lcz.crs} to {iris_paris.crs}...')
        paris_lcz_reproj = paris_lcz.to_crs(iris_paris.crs)
    else:
        paris_lcz_reproj = paris_lcz.copy()
    
    # Identify IRIS column
    iris_col = None
    for col in iris_paris.columns:
        if 'iris' in col.lower() or 'code' in col.lower():
            iris_col = col
            break
    
    if iris_col is None:
        print('❌ Could not identify IRIS column. Using first column as ID.')
        iris_col = iris_paris.columns[0]
    
    print(f'\n🔑 Using column \'{iris_col}\' as IRIS identifier')
    
    # Spatial join: find which IRIS each LCZ zone belongs to
    print('\n⏳ Performing spatial join (this may take a moment)...')
    lcz_with_iris = gpd.sjoin(
        paris_lcz_reproj[['heat_score', 'geometry']],
        iris_paris[[iris_col, 'geometry']],
        how='left',
        predicate='within'
    )
    
    print(f'✅ Spatial join complete')
    print(f'   - LCZ zones with IRIS assignment: {lcz_with_iris[iris_col].notna().sum():,}')
    
    # Aggregate heat scores by IRIS
    print('\n📊 Aggregating heat scores by IRIS...')
    heat_by_iris = lcz_with_iris.groupby(iris_col).agg({
        'heat_score': ['mean', 'median', 'std', 'min', 'max', 'count']
    }).round(2)
    
    # Flatten column names
    heat_by_iris.columns = ['_'.join(col).strip() for col in heat_by_iris.columns.values]
    heat_by_iris = heat_by_iris.reset_index()
    heat_by_iris.columns = ['iris_code', 'avg_heat_score', 'median_heat_score', 
                             'std_heat_score', 'min_heat_score', 'max_heat_score', 
                             'lcz_zones_count']
    
    print(f'\n✅ Computed heat scores for {len(heat_by_iris):,} IRIS zones')
    print(f'\n📈 Heat Score Statistics:')
    print(f'   - Mean heat score: {heat_by_iris["avg_heat_score"].mean():.2f}')
    print(f'   - Max average heat score: {heat_by_iris["avg_heat_score"].max():.2f}')
    print(f'   - Min average heat score: {heat_by_iris["avg_heat_score"].min():.2f}')
    
    print(f'\n📊 Top 10 hottest IRIS zones:')
    print(heat_by_iris.nlargest(10, 'avg_heat_score')[['iris_code', 'avg_heat_score', 'lcz_zones_count']])
    
    print(f'\n❄️ Top 10 coolest IRIS zones:')
    print(heat_by_iris.nsmallest(10, 'avg_heat_score')[['iris_code', 'avg_heat_score', 'lcz_zones_count']])
else:
    print('⚠️  Cannot compute heat scores: IRIS or LCZ data not available')


COMPUTING AVERAGE HEAT SCORE PER IRIS

🔄 Reprojecting LCZ data from EPSG:2154 to EPSG:4326...

🔑 Using column 'iris' as IRIS identifier

⏳ Performing spatial join (this may take a moment)...
✅ Spatial join complete
   - LCZ zones with IRIS assignment: 5,245

📊 Aggregating heat scores by IRIS...

✅ Computed heat scores for 937 IRIS zones

📈 Heat Score Statistics:
   - Mean heat score: 7.58
   - Max average heat score: 9.50
   - Min average heat score: 0.00

📊 Top 10 hottest IRIS zones:
    iris_code  avg_heat_score  lcz_zones_count
560      5920            9.50                2
210      3704            9.12                8
6        0202            9.00               11
7        0203            9.00                9
8        0204            9.00                8
10       0206            9.00                4
12       0302            9.00                3
13       0303            9.00                3
16       0501            9.00               10
17       0502            9.00           

In [31]:
insee_paris[insee_paris['IRIS']== "751072606"][['P22_POP5579','P22_POP80P']]

,P22_POP5579,P22_POP80P
37865,79.550846,45.458107


In [33]:
"""
Cell 8: Calculate Elderly Percentages (with manual column selection)
"""

if insee_data is not None and len(insee_paris) > 0:
    print('='*70)
    print('CALCULATING ELDERLY PERCENTAGES')
    print('='*70)
    
    # IMPORTANT: Update these column names based on your INSEE data
    # These are common patterns - adjust to your actual data
    IRIS_COL = 'IRIS' # IRIS identifier
    TOTAL_POP_COL = ['P22_POP15P'] # Total population - UPDATE THIS
    ELDERLY_55_COL =  ['P22_POP5579','P22_POP80P']
    ELDERLY_80_COL =  ['P22_POP80P']
    LIVING_ALONE_COL = ['P22_POP15P_PSEUL']  # People living alone - UPDATE THIS
    ELDERLY_ALONE_COL = ['P22_POP80P_PSEUL','P22_POP5579_PSEUL']  # Elderly living alone - UPDATE THIS
    
    # Try to auto-detect
    total_candidates = total_pop_cols
    elderly_candidates = elderly_cols
    
    if TOTAL_POP_COL and ELDERLY_55_COL:
        print(f'\n✅ Creating elderly percentage calculations...\n')
        
        # Initialize results dataframe
        elderly_by_iris = insee_paris[[IRIS_COL]].copy()
        
        # Calculate percentage elderly (55+)
        elderly_by_iris['total_population'] = insee_paris[TOTAL_POP_COL]
        elderly_by_iris['elderly_55_plus'] = insee_paris[ELDERLY_55_COL].sum(axis=1)
        elderly_by_iris['pct_elderly_55'] = (
            elderly_by_iris['elderly_55_plus'] / elderly_by_iris['total_population'] * 100
        ).round(2)

        # Calculate percentage elderly (55+) living alone
        elderly_by_iris['elderly_55_plus_alone'] = insee_paris[ELDERLY_ALONE_COL].sum(axis=1)
        elderly_by_iris['pct_elderly_55_alone'] = (
            elderly_by_iris['elderly_55_plus_alone'] / elderly_by_iris['elderly_55_plus'] * 100
        ).round(2)
        
        print(f'📊 Elderly (55+) Statistics:')
        print(f'   - Mean % elderly: {elderly_by_iris["pct_elderly_55"].mean():.2f}%')
        print(f'   - Max % elderly: {elderly_by_iris["pct_elderly_55"].max():.2f}%')
        print(f'   - Min % elderly: {elderly_by_iris["pct_elderly_55"].min():.2f}%')
        print(f'   - Mean % elderly alone: {elderly_by_iris["pct_elderly_55_alone"].mean():.2f}%')
        print(f'   - Max % elderly alone: {elderly_by_iris["pct_elderly_55_alone"].max():.2f}%')
        print(f'   - Min % elderly alone: {elderly_by_iris["pct_elderly_55_alone"].min():.2f}%')
        
        print(f'\n🔟 IRIS with highest elderly population:')
        print(elderly_by_iris.nlargest(10, 'pct_elderly_55')[
            [IRIS_COL, 'total_population', 'elderly_55_plus', 'pct_elderly_55']
        ])

        print(f'\n🔟 IRIS with highest elderly alon population:')
        print(elderly_by_iris.nlargest(10, 'pct_elderly_55_alone')[
            [IRIS_COL, 'total_population', 'elderly_55_plus_alone', 'pct_elderly_55_alone']
        ])
    else:
        print(f'⚠️  Could not identify required columns.')
        print(f'   Total pop column: {TOTAL_POP_COL}')
        print(f'   Elderly 55+ column: {ELDERLY_55_COL}')
        print(f'\n   Please inspect the data and update column references in this cell.')
else:
    print('⚠️  INSEE Paris data not available.')


CALCULATING ELDERLY PERCENTAGES

✅ Creating elderly percentage calculations...

📊 Elderly (55+) Statistics:
   - Mean % elderly: 32.87%
   - Max % elderly: 55.27%
   - Min % elderly: 0.00%
   - Mean % elderly alone: 38.87%
   - Max % elderly alone: 100.00%
   - Min % elderly alone: 0.00%

🔟 IRIS with highest elderly population:
            IRIS  total_population  elderly_55_plus  pct_elderly_55
37865  751072606        226.195528       125.008953           55.27
38702  751208009       1457.355573       792.500361           54.38
37847  751062311       1436.042623       761.079840           53.00
37891  751082905        298.614551       157.867800           52.87
38085  751124614       3331.837774      1726.811659           51.83
38404  751166223         81.125729        41.738426           51.45
38153  751135025       1770.008397       903.673775           51.05
37833  751062202       1241.417230       623.157450           50.20
37892  751082906         10.637293         5.318646       

In [37]:
"""
Cell 10: Combine Heat Exposure and Vulnerability Data
"""

print('='*70)
print('COMBINING HEAT EXPOSURE AND VULNERABILITY METRICS')
print('='*70)

# Merge all three datasets
if 'heat_by_iris' in locals() and 'elderly_by_iris' in locals():
    # Rename columns for consistency
    heat_by_iris_clean = heat_by_iris.rename(columns={'iris_code': iris_col})
    elderly_by_iris_clean = elderly_by_iris.rename(columns={iris_col_insee: iris_col})
    
    # Merge
    paris_iris_combined = heat_by_iris_clean.merge(
        elderly_by_iris_clean[[iris_col, 'pct_elderly_55', 'total_population']],
        on=iris_col,
        how='left'
    )
    
    if 'elderly_alone_by_iris' in locals():
        elderly_alone_clean = elderly_alone_by_iris.rename(columns={iris_col_insee: iris_col})
        paris_iris_combined = paris_iris_combined.merge(
            elderly_alone_clean[[iris_col, 'pct_elderly_55_alone']],
            on=iris_col,
            how='left'
        )
    
    print(f'\n✅ Combined dataset created with {len(paris_iris_combined):,} IRIS zones')
    print(f'\n📊 Dataset structure:')
    print(paris_iris_combined.head(10))
    
    print(f'\n📈 Summary Statistics:')
    print(paris_iris_combined.describe())
else:
    print('⚠️  Missing intermediate datasets. Please run all previous cells.')


COMBINING HEAT EXPOSURE AND VULNERABILITY METRICS

✅ Combined dataset created with 937 IRIS zones

📊 Dataset structure:
   iris  avg_heat_score  median_heat_score  std_heat_score  min_heat_score  \
0  0101            8.36                9.0            1.43               5   
1  0102            3.50                3.5            2.12               2   
2  0103            8.00                9.0            3.11               1   
3  0104            8.50                8.5            2.12               7   
4  0199            0.00                0.0            0.00               0   
5  0201            8.39                9.0            1.85               2   
6  0202            9.00                9.0            0.00               9   
7  0203            9.00                9.0            0.00               9   
8  0204            9.00                9.0            0.00               9   
9  0205            5.00                5.0             NaN               5   

   max_heat_score  lc

In [38]:
"""
Cell 11: Save Processed Data
"""

print('='*70)
print('SAVING PROCESSED DATA')
print('='*70)

if 'paris_iris_combined' in locals():
    # Create output directory
    output_dir = Path('../data/processed')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save individual datasets
    if 'heat_by_iris' in locals():
        heat_file = output_dir / 'paris_iris_heat_scores.csv'
        heat_by_iris.to_csv(heat_file, index=False)
        print(f'\n✅ Saved: {heat_file.name}')
    
    if 'elderly_by_iris' in locals():
        elderly_file = output_dir / 'paris_iris_elderly_pct.csv'
        elderly_by_iris.to_csv(elderly_file, index=False)
        print(f'✅ Saved: {elderly_file.name}')
    
    if 'elderly_alone_by_iris' in locals():
        elderly_alone_file = output_dir / 'paris_iris_elderly_alone_pct.csv'
        elderly_alone_by_iris.to_csv(elderly_alone_file, index=False)
        print(f'✅ Saved: {elderly_alone_file.name}')
    
    # Save combined dataset
    combined_file = output_dir / 'paris_iris_heat_vulnerability.csv'
    paris_iris_combined.to_csv(combined_file, index=False)
    print(f'✅ Saved: {combined_file.name}')
    
    # If IRIS geometry available, save as GeoJSON
    if iris_paris is not None:
        print(f'\n🗺️  Creating GeoJSON with IRIS geometry...')
        iris_col_for_merge = [col for col in iris_paris.columns if 'iris' in col.lower() or 'code' in col.lower()][0]
        
        iris_with_data = iris_paris.merge(
            paris_iris_combined,
            left_on=iris_col_for_merge,
            right_on=iris_col,
            how='left'
        )
        
        geojson_file = output_dir / 'paris_iris_heat_vulnerability.geojson'
        iris_with_data.to_file(geojson_file, driver='GeoJSON')
        print(f'✅ Saved: {geojson_file.name}')
    
    print(f'\n📁 All files saved to: {output_dir}')
else:
    print('⚠️  Combined dataset not available. Please run previous cells.')


SAVING PROCESSED DATA

✅ Saved: paris_iris_heat_scores.csv
✅ Saved: paris_iris_elderly_pct.csv
✅ Saved: paris_iris_heat_vulnerability.csv

🗺️  Creating GeoJSON with IRIS geometry...
✅ Saved: paris_iris_heat_vulnerability.geojson

📁 All files saved to: ../data/processed


In [39]:
"""
Cell 12: Summary and Next Steps
"""

print('='*70)
print('📋 IRIS-LEVEL AGGREGATION SUMMARY')
print('='*70)

print('\n✅ COMPLETED OPERATIONS:')
print('   1. ✓ Computed average heat score per IRIS')
print('   2. ✓ Computed % of elderly persons (55+) per IRIS')
print('   3. ✓ Computed % of elderly living alone per IRIS')

print('\n📊 OUTPUT FILES:')
print('   - paris_iris_heat_scores.csv')
print('   - paris_iris_elderly_pct.csv')
print('   - paris_iris_elderly_alone_pct.csv')
print('   - paris_iris_heat_vulnerability.csv (combined)')
print('   - paris_iris_heat_vulnerability.geojson (with geometries)')

print('\n🚀 NEXT STEPS:')
print('   1. Update Notion documentation with IRIS-level metrics')
print('   2. Integrate this data into the Streamlit app')
print('   3. Update app to display only IRIS-level information')
print('   4. Create vulnerability index combining heat + age factors')
print('   5. Build interactive visualizations for policy makers')

print('\n' + '='*70)
print('✨ IRIS aggregation complete! Ready for app integration.')
print('='*70)


📋 IRIS-LEVEL AGGREGATION SUMMARY

✅ COMPLETED OPERATIONS:
   1. ✓ Computed average heat score per IRIS
   2. ✓ Computed % of elderly persons (65+) per IRIS
   3. ✓ Computed % of elderly living alone per IRIS

📊 OUTPUT FILES:
   - paris_iris_heat_scores.csv
   - paris_iris_elderly_pct.csv
   - paris_iris_elderly_alone_pct.csv
   - paris_iris_heat_vulnerability.csv (combined)
   - paris_iris_heat_vulnerability.geojson (with geometries)

🚀 NEXT STEPS:
   1. Update Notion documentation with IRIS-level metrics
   2. Integrate this data into the Streamlit app
   3. Update app to display only IRIS-level information
   4. Create vulnerability index combining heat + age factors
   5. Build interactive visualizations for policy makers

✨ IRIS aggregation complete! Ready for app integration.
